In [1]:
from  lib.mongo_db  import db_conn
import pandas as pd
import numpy as np
from pyecharts import Bar,configure,Pie,Line
import time
#configure(output_image='svg')


In [2]:
def date_to_time(date):
    timeArray = time.strptime(date, "%Y%m%d")
    time_semp = time.mktime(timeArray)
    return time_semp   # 2013

def time_to_today(timeStamp):
    timeArray = time.localtime(int(timeStamp))
    timeArray =time.strftime("%Y-%m-%d",timeArray)
    return timeArray   # 2013

In [3]:
data=db_conn.report.find({"created_at":{'$gte':date_to_time('20181105'),'$lte':date_to_time('20181115')}},{'_id':0,'sid':1,'cuishou_risk_detection.detection_result':1,'created_at':1})
data1=list(data)

In [4]:
len(data1)

106478

In [5]:
data1_rpt_pd=pd.DataFrame(x for x in data1)

In [6]:
data1_rpt_pd.shape

(106478, 3)

In [7]:
data1_rpt_pd=data1_rpt_pd.fillna({'detection_result': 0})

In [8]:
data1_rpt_pd.head(5)

,created_at,cuishou_risk_detection,sid
0,1.542211e+09,{'detection_result': 2},SIDc5402924ab394a12910b636643b5829c
1,1.542211e+09,{'detection_result': 2},SID92023b1f3402769157251b8318b3fe92
2,1.542211e+09,{'detection_result': 1},SID9be78f389b2100e1a78324ee6c42d318
3,1.542211e+09,{'detection_result': 0},SID4b5326fa62049f6f7845152e95621f31
4,1.542211e+09,{'detection_result': 1},SID46f6806ca92171278913ff75f49a9977


In [9]:
data1_rpt_pd['created_at']=data1_rpt_pd.created_at.map(time_to_today)

In [10]:
detection_result={0:'无催收记录',1:'轻度催收',2:'中度催收',3:'重度催收'}

In [11]:
def get_dict(x):
    try:
        b=x.get('detection_result',0)
    except:
        b=0
    return b

In [12]:
data1_rpt_pd['cuishou_risk_detection']=data1_rpt_pd.cuishou_risk_detection.map(get_dict)

In [13]:
ret_pd=data1_rpt_pd.groupby(['created_at','cuishou_risk_detection'])['sid'].size().reset_index()

In [14]:
ret_pd

,created_at,cuishou_risk_detection,sid
0,2018-11-05,0,2461
1,2018-11-05,1,4449
2,2018-11-05,2,2584
3,2018-11-05,3,262
4,2018-11-06,0,2780
5,2018-11-06,1,4824
6,2018-11-06,2,2395
7,2018-11-06,3,247
8,2018-11-07,0,2732
9,2018-11-07,1,5335


In [15]:
def pri_tu(data):
    bar= Line('day_anly','Theme',renderer ='svg')
    for values in range(4):
        sub_data=data.loc[data.cuishou_risk_detection.values==values,:]
        str1=sub_data.created_at.values.tolist()
        #str1.sort()
        bar.add(detection_result[values],str1,sub_data.sid.values.tolist(),is_more_utils = True,is_smooth=True)
    return bar

In [16]:
pri_tu(ret_pd)